# Dynamic Investment Blueprint

### Goal: Call a stock market API to target companies of interest while instantly eradicating any companies that do not pass validation boundaries from our initial investment strategy.

### Why? Finding new talent to invest in may be tough and time consuming. By validating companies with real-time updated data provided by a third-party company's API, we can save time and focus fundamental investment strategies. Note: achievement of growth or value investing

### This Project: The project that you will read about here is only an illustration of what we can accomplish with my program. This program may be re-used with any technical parameters of interest and will output an analysis that can be used for projections and straight forward visualization diagrams. In this example we will call the API and validate any companies that fit our 'growth' investing scope (boundaries). Growth investing methods look for smaller companies that have potential for high growth in the future. Although this method is prone to possible loss, if we win there is a much better payout. So lets hundred-fold our investment!

### Part 1 - Picking a Strategy

#### Importing libraries and packages

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

#### Declare Functions here

##### Function to import a javascript formatted URL into variable

In [34]:
# This code was copied from,
# Credit: https://financialmodelingprep.com/developer/docs/companies-key-stats-free-api/#Python
#!/usr/bin/env python

try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import json

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

##### Function to convert javascript into a pandas dataframe

In [35]:
def json_to_empty_dataframe(dictionary):
    columns = ['Company']
    # Create a list of the dictionary keys
    for keys in dictionary:
        columns.append(keys)
    
    # Create a dataframe with the columns
    initial_dataframe = pd.DataFrame(columns=columns)
    
    return initial_dataframe
    
#def json_fill_dataframe

##### Functions for Indicator Calculations

In [38]:
# Use this function to calculate eps ratio
#def earnings_per_share_ratio(total_profit, number_of_shares):

# Use this function to find the growth of the eps
#def earnings_per_share_growth(eps):

##### Other useful functions

In [175]:
# Function which extracts a big list of companies
def company_list(stock_list):
    # stock_list contains 'symbol', 'name', and 'price'
    final_list = []
    counter = 0
    # Loop through all stock name and append them
    for i in stock_list:
        for key in stock_list[counter]:
            #if key == 'price':
                if (stock_list[counter])['price'] < 0.5:
                    final_list.append((stock_list[counter])['symbol'])
        counter = counter + 1
        
    return final_list

# Function to input the stock symbol into the url so we can call the API
def name_API_company_profile(symbol, url):
    return url[:54] + symbol + url[54:]

def name_API_income_statement(symbol, url):
    return url[:69] + symbol + url[69:]

##### Call API from resource: financialmodelingprep.com

##### Get a ticker list of all companies in the stock market

In [166]:
# Import companies from API
url_stocks = ("https://financialmodelingprep.com/api/v3/company/stock/list")
stock_list = get_jsonparsed_data(url_stocks)

hierarchy = stock_list["symbolsList"]
# Convert json data into a list of all stocks
all_stocks = company_list(hierarchy)
print(all_stocks)

['HK', 'HK', 'HK', 'WFT', 'WFT', 'WFT', 'TRQ', 'TRQ', 'TRQ', 'AMR', 'AMR', 'AMR', 'WIN', 'WIN', 'WIN', 'SPN', 'SPN', 'SPN', 'SN', 'SN', 'SN', 'UPL', 'UPL', 'UPL', 'GSAT', 'GSAT', 'GSAT', 'TTPH', 'TTPH', 'TTPH', 'HMNY', 'HMNY', 'HMNY', 'EPE', 'EPE', 'EPE', 'GST', 'GST', 'GST', 'AKAO', 'AKAO', 'AKAO', 'EVEP', 'EVEP', 'EVEP', 'FRED', 'FRED', 'FRED', 'ENRJ', 'ENRJ', 'ENRJ', 'APHB', 'APHB', 'APHB', 'SGYP', 'SGYP', 'SGYP', 'EGLT', 'EGLT', 'EGLT', 'DSS', 'DSS', 'DSS', 'ONVO', 'ONVO', 'ONVO', 'ASNA', 'ASNA', 'ASNA', 'CPHI', 'CPHI', 'CPHI', 'ACET', 'ACET', 'ACET', 'FSNN', 'FSNN', 'FSNN', 'BRS', 'BRS', 'BRS', 'SRRA', 'SRRA', 'SRRA', 'SHLD', 'SHLD', 'SHLD', 'PES', 'PES', 'PES', 'EVLV', 'EVLV', 'EVLV', 'ATNM', 'ATNM', 'ATNM', 'CNAT', 'CNAT', 'CNAT', 'AEMD', 'AEMD', 'AEMD', 'ADXS', 'ADXS', 'ADXS', 'EMES', 'EMES', 'EMES', 'FCEL', 'FCEL', 'FCEL', 'ZN', 'ZN', 'ZN', 'SIGM', 'SIGM', 'SIGM', 'INUV', 'INUV', 'INUV', 'CLD', 'CLD', 'CLD', 'INSY', 'INSY', 'INSY', 'ARLZ', 'ARLZ', 'ARLZ', 'RVLT', 'RVLT', 'RVLT

#### Create an empty dataframe with relevant columns of multiple stocks

In [176]:
# Identify our data identifiers
ticker = "AAPL"
url_company_profile = ("https://financialmodelingprep.com/api/company/profile/?datatype=json") #54
url_income_statement = ("https://financialmodelingprep.com/api/v3/financials/income-statement/?period=quarter") #68

# Add ticker symbol into data URL - this extracts data from the website reference
api_call = name_API_company_profile(ticker, url_company_profile)
print(api_call)


# Take the website data from json into a pandas dataframe
js = get_jsonparsed_data(api_call)
stock_data = js[ticker]
print(stock_data)

test = json_to_empty_dataframe(stock_data)
test
#for key in stock_data:
 #   print(key)
 

https://financialmodelingprep.com/api/company/profile/AAPL?datatype=json
{'Price': 219.46, 'Beta': '1.139593', 'VolAvg': '36724977', 'MktCap': '1090573773990.00', 'LastDiv': '2.92', 'Range': '142-233.47', 'Changes': -0.01, 'ChangesPerc': '(-0.00%)', 'companyName': 'Apple Inc.', 'exchange': 'Nasdaq Global Select', 'industry': 'Computer Hardware', 'website': 'http://www.apple.com', 'description': 'Apple Inc is designs, manufactures and markets mobile communication and media devices and personal computers, and sells a variety of related software, services, accessories, networking solutions and third-party digital content and applications.', 'CEO': 'Timothy D. Cook', 'sector': 'Technology', 'date_is_filed': None, 'date_bs_filed': None, 'date_cs_filed': None, 'image': 'https://financialmodelingprep.com/images-New-jpg/AAPL.jpg'}


,Company,Price,Beta,VolAvg,MktCap,LastDiv,Range,Changes,ChangesPerc,companyName,exchange,industry,website,description,CEO,sector,date_is_filed,date_bs_filed,date_cs_filed,image


##### Create a dataframe of one stock we want to look deeper into, this is where all the analysis of single stocks will come from

In [178]:
url_income_statement = ("https://financialmodelingprep.com/api/v3/financials/income-statement/?period=quarter") #68

# Add ticker symbol into data URL - this extracts data from the website reference
api_call = name_API_income_statement(ticker, url_income_statement)

